## Train Model

### Configure Spark for Your Notebook
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`

In [1]:
import os

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
conf = '--conf spark.cores.max=1 --conf spark.executor.memory=512m'
packages = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'
jars = '--jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar'
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + conf \
  + ' ' + packages \
  + ' ' + jars \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

--master local[1] --conf spark.cores.max=1 --conf spark.executor.memory=512m --packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1 --jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar --py-files /root/lib/jpmml.py pyspark-shell


### Import Spark Libraries

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

### Configure Spark for Your Notebook
* You may need to Reconnect and/or Restart the Kernel to pick up changes.
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`zzaa

### Create Spark Session
This may take a minute or two.  Please be patient.

In [3]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.getOrCreate()

### Read Data from Public S3 Bucket
* AWS credentials are not needed.
* We're asking Spark to infer the schema
* The data has a header
* Using `bzip2` because it's a splittable compression file format

In [4]:
df = spark_session.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")

print(df.head())

Row(id=5731498, name='A 2-bdrm house in Plaka of Athens', space='Ideally located a unique house in a very peaceful neighborhood of Plaka, near Acropolis. It is a traditional house in the heart of the historical center of Athens, in Plaka. The kitchen is fully equipped with oven, fridge with freezer. Cutlery, dishes and pans, kettle, espresso coffee maker (espresso capsules are provided), toaster. There is also a vacuum cleaner and a laundry machine. One big closet will make your stay more comfortable. Bed linen, towels and bath amenities are provided. Moreover, the apartment is fully airconditioned. The apartment is very close to a greek traditional tavernas, a pharmacy, banks and public transport.  Airport or any other transport is available upon demand at an additional but very reasonable cost. ', price='120.0', bathrooms='1.0', bedrooms='2.0', room_type='Entire home/apt', square_feet=None, host_is_super_host='0.0', city='Athina', state=None, cancellation_policy='moderate', security_

In [5]:
print(df.count())

198576


### Clean, Filter, and Summarize the Data

In [8]:
print(df_final.count())

151864


In [7]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark_session.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+
|summary|      square_feet|             price|          bedrooms|         bathrooms|     cleaning_fee|
+-------+-----------------+------------------+------------------+------------------+-----------------+
|  count|           151864|            151864|            151864|            151864|           151864|
|   mean|545.5920823895063|131.00769109202972|1.3336998893747036|1.1988786019069695|37.25118527103198|
| stddev|363.2346181084825| 89.59372969879449|0.8460907193971746|0.4836515839573332|  42.625502170779|
|    min|            104.0|              50.0|               0.0|               0.5|              0.0|
|    max|          32292.0|             750.0|              10.0|               8.0|            700.0|
+-------+-----------------+------------------+------------------+------------------+-----------------+



In [9]:
print(df_final.schema)

StructType(List(StructField(id,IntegerType,true),StructField(city,StringType,true),StructField(state,StringType,true),StructField(space,StringType,true),StructField(price,DoubleType,true),StructField(bathrooms,DoubleType,true),StructField(bedrooms,DoubleType,true),StructField(room_type,StringType,true),StructField(host_is_super_host,StringType,true),StructField(cancellation_policy,StringType,true),StructField(security_deposit,DoubleType,true),StructField(price_per_bedroom,StringType,true),StructField(number_of_reviews,DoubleType,true),StructField(extra_people,DoubleType,true),StructField(instant_bookable,StringType,true),StructField(cleaning_fee,DoubleType,true),StructField(review_scores_rating,DoubleType,true),StructField(square_feet,DoubleType,true)))


In [10]:
# Most popular cities

spark_session.sql("""
    select 
        state,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by state
    order by count(*) desc
""").show()

+------+-----+------------------+---------+
| state|   ct|         avg_price|max_price|
+------+-----+------------------+---------+
| Other|87467|122.00503046863389|    750.0|
|    NY|22899| 145.9446264028997|    750.0|
|    CA|20750|157.40173493975902|    750.0|
|Berlin| 6034|  80.6433543254889|    650.0|
|    IL| 3552|141.46903153153153|    690.0|
|    TX| 3108|195.25611325611325|    750.0|
|    WA| 2700| 131.4962962962963|    750.0|
|    DC| 2590|136.64015444015445|    720.0|
|    OR| 1954|114.02661207778915|    700.0|
|London|  810|108.84444444444445|    600.0|
+------+-----+------------------+---------+



In [11]:
# Most expensive popular cities

spark_session.sql("""
    select 
        city,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by city
    order by avg(price) desc
""").filter("ct > 25").show()

+-------------------+---+------------------+---------+
|               city| ct|         avg_price|max_price|
+-------------------+---+------------------+---------+
|         Palm Beach| 26| 348.7692307692308|    701.0|
|        Watsonville| 38| 313.3157894736842|    670.0|
|             Malibu|136| 280.9852941176471|    750.0|
|             Avalon| 38|262.42105263157896|    701.0|
|           Capitola| 35|             246.4|    650.0|
|           Tamarama| 72|             238.5|    750.0|
|    Manhattan Beach|109|232.10091743119267|    700.0|
|Rancho Palos Verdes| 39|230.02564102564102|    750.0|
|       Avalon Beach| 38|229.60526315789474|    620.0|
|            Newport| 52| 223.8653846153846|    750.0|
|      Darling Point| 29|221.51724137931035|    623.0|
|        Middle Park| 34| 212.7941176470588|    671.0|
|            Balmain| 55|212.56363636363636|    712.0|
|        North Bondi|180|206.68333333333334|    750.0|
|             Bronte|144|203.70833333333334|    750.0|
|        Q

### Define Continous and Categorical Features

In [12]:
continuous_features = ["bathrooms", \
                       "bedrooms", \
                       "security_deposit", \
                       "cleaning_fee", \
                       "extra_people", \
                       "number_of_reviews", \
                       "square_feet", \
                       "review_scores_rating"]

categorical_features = ["room_type", \
                        "host_is_super_host", \
                        "cancellation_policy", \
                        "instant_bookable", \
                        "state"]

### Continuous Features

In [13]:
continuous_feature_assembler = VectorAssembler(inputCols=continuous_features, outputCol="unscaled_continuous_features")

continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", \
                                           withStd=True, withMean=False)

### Categorical Features


In [14]:
categorical_feature_indexers = [StringIndexer(inputCol=x, \
                                              outputCol="{}_index".format(x)) \
                                for x in categorical_features]

categorical_feature_one_hot_encoders = [OneHotEncoder(inputCol=x.getOutputCol(), \
                                                      outputCol="oh_encoder_{}".format(x.getOutputCol() )) \
                                        for x in categorical_feature_indexers]

### Assemble Features

In [15]:
feature_cols_lr = [x.getOutputCol() \
                   for x in categorical_feature_one_hot_encoders]
feature_cols_lr.append("scaled_continuous_features")

feature_assembler_lr = VectorAssembler(inputCols=feature_cols_lr, \
                                       outputCol="features_lr")

### Train LR Model


In [16]:
linear_regression = LinearRegression(featuresCol="features_lr", \
                                     labelCol="price", \
                                     predictionCol="price_prediction", \
                                     maxIter=10, \
                                     regParam=0.3, \
                                     elasticNetParam=0.8)

estimators_lr = \
  [continuous_feature_assembler, continuous_feature_scaler] \
  + categorical_feature_indexers + categorical_feature_one_hot_encoders \
  + [feature_assembler_lr] + [linear_regression]

pipeline = Pipeline(stages=estimators_lr)

pipeline_model = pipeline.fit(df_final)

print(pipeline_model)

PipelineModel_4beabd3a801bb2c96921


## Export the Spark ML Pipeline

In [17]:
from jpmml import toPMMLBytes

model = toPMMLBytes(spark_session, df_final, pipeline_model)
with open('model.spark', 'wb') as fh:
    fh.write(model)

In [18]:
%%bash

ls -l model.spark

-rw-rw-r-- 1 root root 7275 Sep 20 14:18 model.spark
